In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time

In [2]:
# user = get_upstox_user("sonupal129@gmail.com")
# symbol = Symbol.objects.get(symbol="ashokley")
# # print(user.get_ohlc(user.get_instrument_by_symbol('NSE_EQ', 'bel'), OHLCInterval.Minute_1, datetime.strptime('12/12/2019', '%d/%m/%Y').date(), datetime.strptime('12/12/2019', '%d/%m/%Y').date()))
# user.get_master_contract("NSE_EQ")
# user.get_master_contract("NSE_INDEX")
# # cache_candles_data(user, symbol)
# # get_stock_current_candle("ashokley")

In [29]:
redis_cache = caches["redis"]
redis_cache.get("ccl")

[{'timestamp': '1576228140000',
  'open': '183.00',
  'high': '183.00',
  'low': '183.00',
  'close': '183.00',
  'volume': '0'},
 {'timestamp': '1576228200000',
  'open': '183.00',
  'high': '183.00',
  'low': '183.00',
  'close': '183.00',
  'volume': '0'}]

In [4]:
# def function_caller(start_time, end_time, func, *args, **kwargs):
#     """Call function on custom time with interval functionality using celery periodic task"""
#     current_time = datetime.now().time()
#     if start_time < current_time < end_time:
#         func(*args, **kwargs)

In [5]:
# def func(a,b):
#     a= 3
#     b = 4
#     print(a+b)
    
# start = time(9,15)
# end = time(15,30)
    
# function_caller(start,end, func, 3,4)

In [36]:
get_stock_live_data("ccl")

,candle_type,open_price,high_price,close_price,low_price,volume,atp,total_buy_quantity,total_sell_quantity,lower_circuit,upper_circuit,bids,asks,date
0,M5,183.00,183.00,183.00,182.65,None,None,None,None,None,None,NaN,NaN,2019-12-13 14:50:00


In [31]:
redis_cache.get("ccl")

[{'timestamp': '1576228620000',
  'open': '182.95',
  'high': '182.95',
  'low': '182.95',
  'close': '182.95',
  'volume': '0'},
 {'timestamp': '1576228680000',
  'open': '182.95',
  'high': '183.00',
  'low': '182.90',
  'close': '183.00',
  'volume': '416'}]

In [32]:
datetime.fromtimestamp(1576228680)

datetime.datetime(2019, 12, 13, 14, 48)